In [2]:
from ClfTemplates import get_all_template_path,sample_template,sample_template_path
from ClfUse import showClfResult, get_samples, labeledDataFromJson,preprocess
import warnings
warnings.filterwarnings("ignore")

In [2]:

banklist = ['SBIUPI', 'SBIOTP', 'KOTKBK', 'MYACCT', 'BOBTXN',
            'RBISAY', 'CSHBCK', 'ATMSBI', 'UMOBIL', 'PAYTMB']

template = '/data-0/tigergraph/template'
all_templates = get_all_template_path(template_dir = template)
sampled_template_path = sample_template_path(all_templates)
sampled_template_df = sample_template(sampled_template_path,1)

# 全部银行模版中，只取top10打标签；
count_bank = sampled_template_df.groupby('bank').count()
top_10_bank = count_bank.sort_values(by ='sms').head(10).index.tolist()
top_10_sampled_template_df = sampled_template_df[sampled_template_df.bank.isin(top_10_bank)]
top_10_sampled_template_df['unique_template_id'] = top_10_sampled_template_df.bank + '_' + top_10_sampled_template_df.cluster_id

# top10打标签结果；
top_10_train_path = '/home/qibo/项目2_sms/综合/bank-top-10-data/sms_top_10_bank.json'
last_40_train_path = '/home/qibo/项目2_sms/综合/bank-top-10-data/last40bank.json'

labeled_df = labeledDataFromJson(top_10_train_path)
labeled_df.loc[labeled_df.label == 'error', 'label'] = ['流水转出确认支付密码']

# 拼接到一起；
top_10_sampled_template_df['label'] = labeled_df.label.tolist()


total templates among all the banks:1641


In [3]:
sms = get_samples(banklist, sampled_template_path)
showClfResult(sms, top_10_sampled_template_df)

processing sentence num:0
new_instance:Please click on below link to set paytm passcode http://m.p-y.tm/cpsc

此短信来自于模版:PAYTMB_11

元组第三个代表模版ID:(1.0, '其他', 'PAYTMB_11')


'其他'

In [14]:
all_temp = top_10_sampled_template_df.unique_template_id.tolist()
'PAYTMB_2' in all_temp

True

#### 错误识别， 模版补充， 重新训练；

In [6]:
sms_try = 'dear customer you have received rs. 5020. 00 via imps in your account xxxxxxxx5993 from bheemavarapu manoj k (xxxxxx5993). ref no: 823009082098. queries write to us at http://m. p-y. tm/care'

showClfResult(sms_try, top_10_sampled_template_df)

processing sentence num:0
 dear customer you have received rs. 5020. 00 via imps in your account xxxxxxxx5993 from bheemavarapu manoj k (xxxxxx5993). ref no: 823009082098. queries write to us at http://m. p-y. tm/care

元组第三个代表模版ID:(0.49210635006764225, '流水-转出', 'PAYTMB_2')


'流水-转出'

### 周三下午； 打标签 银行remaining

In [4]:
from ClfTemplates import get_all_template_path,sample_template,sample_template_path
from ClfUse import showClfResult, get_samples, labeledDataFromJson

template = '/data-0/tigergraph/template'
all_templates = get_all_template_path(template_dir = template)
sampled_template_path = sample_template_path(all_templates)
sampled_template_df = sample_template(sampled_template_path, num_samples=1)

# 全部银行模版中，取last40打标签；
count_bank = sampled_template_df.groupby('bank').count()

remaining_bank_list = count_bank.sort_values(by ='sms').iloc[10:].index.tolist()
last40_sampled_template_df = sampled_template_df[sampled_template_df.bank.isin(remaining_bank_list)]
last40_sampled_template_df['unique_template_id'] = last40_sampled_template_df.bank + '_' + last40_sampled_template_df.cluster_id
last40_sampled_template_df['sms'] = last40_sampled_template_df.unique_template_id + ' ' + last40_sampled_template_df.sms
last40_sampled_template_df = preprocess(last40_sampled_template_df)


total templates among all the banks:1641
processing sentence num:0


In [5]:
# 打完标签上传上来；
last_40_train_path = '/home/qibo/项目2_sms/综合/bank-top-10-data/last40bank.json'
labeled_df = labeledDataFromJson(last_40_train_path)
# 拼接到一起；
last40_sampled_template_df['label'] = labeled_df.label.tolist()
last40_sampled_template_df.label = last40_sampled_template_df.label.apply(lambda x: x[0] if isinstance(x, list) else x)

# md, 我分类的预处理没有对这个括号做处理，然而NER中需要这一步操作，分类中 必须要保证 模版中的sms处理方式与instance一致，即不处理为最好。
last40_sampled_template_df.new_sms = [i.strip().replace('(', ' (') for i in last40_sampled_template_df.new_sms.tolist()]

## 采样100个 用来最test

In [29]:
#将上面的弄到本地打标签
to_tag3 = last40_sampled_template_df[last40_sampled_template_df.label!='sms_other']
sms = to_tag3.sample(100).sms.tolist()
with open("last40_bank_TEST100_NER.txt", "w") as f:
    for i in sms:
        f.write(i)

In [9]:
Last40Banklist = last40_sampled_template_df.bank.unique()

In [10]:
sms = get_samples(Last40Banklist, sampled_template_path)
showClfResult(sms, last40_sampled_template_df)

processing sentence num:0
new_instance:Dear Customer Your Aadhar number updation request has been registered vide SR501756666 on 18-OCT-17. It will be completed within 7 working days.

此短信来自于模版:ICICIB_78

元组第三个代表模版ID:(0.5198714756890587, 'sms_other', 'ICICIB_78')


'sms_other'

In [11]:
# 有个别错误标签；
revise = ['sms_other', '交易流水＿转账', 'sms_other','交易流水＿转账','sms_other','信用卡＿还款提醒','sms_other','sms_other','交易流水＿转账']
error_idx = last40_sampled_template_df[last40_sampled_template_df.label.isin(['error'])].index
last40_sampled_template_df.loc[error_idx,'label'] = revise

In [12]:
# 先取一半打标签；
half_bank = last40_sampled_template_df.bank.unique()[:9]
count_bank = last40_sampled_template_df.groupby('bank').count()
half_bank = count_bank.sort_values(by ='sms')[:18].index.tolist()


to_tag2 = last40_sampled_template_df[last40_sampled_template_df.label!='sms_other']
to_tag2 = to_tag2[to_tag2.bank.isin(half_bank)]
to_tag2.new_sms = to_tag2.label + ' ' + to_tag2.new_sms

In [17]:
#将上面的弄到本地打标签
with open("last40_bank_template_NER.txt", "w") as f:
    for i in to_tag2.new_sms.tolist():
        f.write(i)

In [ ]:
scp tigergraph@192.168.20.241:/home/tigergraph/generate_template_qb_test/last40_bank_template_NER.txt /Users/cashbus/Documents/工作/

In [ ]:
last40_bank_template_NER.txt